In [1]:
! pip install -qU langchain-openai langchain langchain_community langchainhub
! pip install chromadb==0.5.3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 45.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 399.9/399.9 kB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 292.2/292.2 kB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.0/383.0 kB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 37.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141

In [3]:
from langchain import hub as langchain_hub
from langchain.schema import StrOutputParser
from langchain_openai import ChatOpenAI
from langchain.schema.runnable import RunnablePassthrough
from langchain.text_splitter import MarkdownHeaderTextSplitter
from langchain_openai import OpenAIEmbeddings
import os
#from langchain_chroma import Chroma
from langchain_community.vectorstores.chroma import Chroma

## 对数据进行切片，存入向量数据库chroma

In [6]:
# 读取 ./data/data.md 文件作为运维知识库
file_path = os.path.join('data', 'data.md')
with open(file_path, 'r', encoding='utf-8') as file:
    docs_string = file.read()

# Split the document into chunks base on markdown headers.
headers_to_split_on = [
    ("#", "Header 1"),
    ("##", "Header 2"),
    ("###", "Header 3"),
]
text_splitter = MarkdownHeaderTextSplitter(headers_to_split_on=headers_to_split_on)
splits = text_splitter.split_text(docs_string)
print("Length of splits: " + str(len(splits)))
print(splits)

Length of splits: 11
[Document(metadata={'Header 1': '支付系统运维知识库', 'Header 2': '1. 系统监控指标'}, page_content='- **交易量**: 监控每秒交易数，确保系统承载能力。\n- **响应时间**: 监控交易的平均响应时间，确保服务性能。\n- **系统负载**: 监控CPU、内存等资源使用率，避免资源瓶颈。'), Document(metadata={'Header 1': '支付系统运维知识库', 'Header 2': '2. 常见问题与解决方案', 'Header 3': '2.1 交易失败'}, page_content='- **问题描述**: 用户发起支付后，交易未能成功完成。\n- **可能原因**:\n- 网络延迟或中断\n- 支付网关服务异常\n- **解决办法**:\n- 检查网络连接，确保支付服务的网络畅通。\n- 检查支付网关服务状态，重启服务或联系服务提供商。'), Document(metadata={'Header 1': '支付系统运维知识库', 'Header 2': '2. 常见问题与解决方案', 'Header 3': '2.2 响应时间过长'}, page_content='- **问题描述**: 用户支付请求处理时间超过正常范围。\n- **可能原因**:\n- 系统资源不足\n- 数据库查询效率低下\n- 外部服务响应慢\n- **解决办法**:\n- 增加系统资源，如CPU、内存。\n- 优化数据库查询，使用索引，减少复杂查询。\n- 与外部服务提供商沟通，优化接口性能。'), Document(metadata={'Header 1': '支付系统运维知识库', 'Header 2': '2. 常见问题与解决方案', 'Header 3': '2.3 系统宕机'}, page_content='- **问题描述**: 支付系统完全无法访问或服务中断。\n- **可能原因**:\n- 主机硬件故障\n- 系统软件崩溃\n- 网络设备故障\n- **解决办法**:\n- 快速切换到备用服务器。\n- 检查系统日志，定位问题原因。\n- 联系硬件供应商，进行故障排查和修复。'), Document(metadata={'Head

In [7]:
# 将运维知识库的每一块文本向量化（Embedding）
embedding = OpenAIEmbeddings(model="text-embedding-3-small", openai_api_key="sk-T6SlqfUnyFytejvA3c1584F87d6343878232185e26243b1d",openai_api_base="https://api.apiyi.com/v1")
vectorstore = Chroma.from_documents(documents=splits, embedding=embedding,persist_directory="./chroma_langchain_db")
vectorstore.persist()

<ipython-input-7-95e725cfb409>:4: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vectorstore.persist()


In [17]:
# 文本相似性检索
results = vectorstore.similarity_search_with_score(
    "payment_gateway服务",
    k=1,
)
for res, score in results:
    print(f"* [相似性={score:3f}] {res.page_content} [{res.metadata}]")

* [相似性=0.870702] - **payment**: 小张，联系方式：18888888888
- **payment_gateway**: 小王，联系方式：18888888889
- **payment_callback**: 小李，联系方式：18888888890 [{'Header 1': '支付系统运维知识库', 'Header 2': '8. 业务负责人'}]


In [10]:
# 向量检索

results = vectorstore.similarity_search_by_vector_with_relevance_scores(
    embedding=embedding.embed_query("payment 服务是谁维护的？"), k=1
)
for doc, score in results:
    print(f"* [相似性={score:3f}] {doc.page_content} [{doc.metadata}]")

* [相似性=1.088800] - **payment**: 小张，联系方式：18888888888
- **payment_gateway**: 小王，联系方式：18888888889
- **payment_callback**: 小李，联系方式：18888888890 [{'Header 1': '支付系统运维知识库', 'Header 2': '8. 业务负责人'}]


In [12]:
# 向量检索时会把 input Embedding 之后转成向量
from openai import OpenAI
client = OpenAI(
    api_key="sk-T6SlqfUnyFytejvA3c1584F87d6343878232185e26243b1d",
    base_url="https://api.apiyi.com/v1"
)

response = client.embeddings.create(
    input="payment 服务是谁维护的？",
    model="text-embedding-3-small"
)

print(response.data[0].embedding)

[0.013628354296088219, -0.03324894234538078, 0.05509673431515694, 0.008762978948652744, -0.004756004083901644, -0.002200687536969781, -0.015205956064164639, 0.051093071699142456, -0.006532462779432535, -0.05854359269142151, 0.09242887794971466, 0.0045969183556735516, 0.010479780845344067, -0.04435843974351883, -0.024923449382185936, 0.007172120735049248, -0.0306240264326334, 0.038843464106321335, -0.041998665779829025, -0.017658529803156853, 0.0033822315745055676, 0.03955934941768646, 0.0053525762632489204, -0.012216467410326004, -0.04157443717122078, 0.013628354296088219, -0.048759814351797104, -0.04176003485918045, 0.022431105375289917, -0.06421765685081482, 0.06363434344530106, -0.029722539708018303, 0.05615730583667755, 0.030385397374629974, 0.03812757506966591, -0.0027160595636814833, -0.00542880455031991, -0.029669512063264847, 0.017605500295758247, -0.006834063213318586, -0.02301442064344883, -0.05578610673546791, -0.003417031606659293, -0.022775791585445404, -0.0061745196580886

In [21]:
# 使用运维知识库进行检索生成
retriever = vectorstore.as_retriever()
# 使用 LangChain 提供的 RAG Prompt，来源于：https://smith.langchain.com/hub/rlm/rag-prompt?organizationId=989ad331-949f-4bac-9694-660074a208a7
"""
这是 RAG Prompt 原文：

human

You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.

Question: {question}

Context: {context}

Answer:
"""
prompt = langchain_hub.pull("rlm/rag-prompt")

print(prompt)

def format_docs(docs):
    print("匹配到的运维知识库片段：\n", "\n\n".join(doc.page_content for doc in docs))
    return "\n\n".join(doc.page_content for doc in docs)

llm = ChatOpenAI(model="gpt-4o-mini",
    api_key="sk-T6SlqfUnyFytejvA3c1584F87d6343878232185e26243b1d",
    base_url="https://api.apiyi.com/v1"
    )
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

# 问题将会被 Embedding 转成向量，然后再从向量数据库检索
res = rag_chain.invoke("payment 服务是谁维护的？")
# RAG 无法回答的问题
# res = rag_chain.invoke("谁负责的服务最多？")
print("\n\nLLM 回答：", res)

/usr/local/lib/python3.10/dist-packages/langsmith/client.py:323: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


input_variables=['context', 'question'] input_types={} partial_variables={} metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'} messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"), additional_kwargs={})]
匹配到的运维知识库片段：
 - **payment**: 小张，联系方式：18888888888
- **payment_gateway**: 小王，联系方式：18888888889
- **payment_callback**: 小李，联系方式：18888888890

- **问题描述**: 用户发起支付后，交易未能成功完成。
- **可能原因**:
- 网络延迟或中断
- 支付网关服务异常
- **解决办法**:
- 检查网络连接，确保支付服务的网络畅通。
- 检查支付网关服务状态，重启服务或联系服务提供商。

- **问题描述**: 支付系统完全无法访问或服务中断。
- **可能原因**:
- 主机

In [24]:
res = rag_chain.invoke("谁负责的服务最多？")
print("\n\nLLM 回答：", res)

匹配到的运维知识库片段：
 - **交易量**: 监控每秒交易数，确保系统承载能力。
- **响应时间**: 监控交易的平均响应时间，确保服务性能。
- **系统负载**: 监控CPU、内存等资源使用率，避免资源瓶颈。

- **问题描述**: 用户支付请求处理时间超过正常范围。
- **可能原因**:
- 系统资源不足
- 数据库查询效率低下
- 外部服务响应慢
- **解决办法**:
- 增加系统资源，如CPU、内存。
- 优化数据库查询，使用索引，减少复杂查询。
- 与外部服务提供商沟通，优化接口性能。

- **问题描述**: 支付系统完全无法访问或服务中断。
- **可能原因**:
- 主机硬件故障
- 系统软件崩溃
- 网络设备故障
- **解决办法**:
- 快速切换到备用服务器。
- 检查系统日志，定位问题原因。
- 联系硬件供应商，进行故障排查和修复。

- **发现问题**: 监控系统发现异常指标。
- **快速响应**: 立即通知运维团队，进行初步诊断。
- **问题定位**: 通过日志分析，定位问题原因。
- **解决方案**: 根据问题类型，执行相应的解决方案。
- **问题解决**: 确认问题解决，恢复正常服务。
- **后续跟进**: 记录问题处理过程，进行事后分析和总结。


LLM 回答： 根据提供的上下文，负责服务最多的可能是运维团队，因为他们需要监控系统、处理异常指标并快速响应问题。


rag缺点：
  * 文本分片会把上下文逻辑截断，导致无法回答内在逻辑关联问题
  * 只能回答显式的检索问题，无法回答全局性或者总结性的问题